# S&P 600 Wikipedia Scrape

In [ ]:
''' This workbok scrapes the S&P 600 and then uses pandas to turn into a dataframe and look at different fudamental and
price features of a given security:
S&P 600 landing page: 
https://en.wikipedia.org/wiki/List_of_S%26P_600_companies'''
# import libraries
import pandas as pd # excel for python
import requests as r # ability to bring in data
import os # brings in os functions
import regex
from bs4 import BeautifulSoup # ability to parse a web page for the data that we need
import yfinance as yf #used to request a securities feature data

In [ ]:
#get data from page
headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 \
           (KHTML, like Gecko) \
           Chrome/47.0.2526.106 Safari537.36'}
url='https://en.wikipedia.org/wiki/List_of_S%26P_600_companies'
sp600=r.get(url,headers=headers)
sp600.status_code # provides status code

In [ ]:
sp600.request.headers # check headers

In [ ]:
# lets make some soup
soup = BeautifulSoup(sp600.content, 'html.parser')
tables = soup.find_all('table', attrs={'class': 'wikitable'})
sp600cl = tables[0]
sp600changes = tables[1]
# sp600cl

In [ ]:
def parse_table(html_table) -> pd.DataFrame:
    columns =[]
    rows = []
    for column in html_table.find_all('th'): # 'th' table headers
        columns.append(column.text)
    for row in html_table.find_all('tr'): # 'tr' table rows
        row_to_add =[]
        for cell in row.find_all('td'): # 'td' table data
            row_to_add.append(cell.text)
        rows.append(row_to_add)
    df = pd.DataFrame(rows,columns=columns)
    return df

In [ ]:
# clean data, this will be turned into a function at a later time.
sp600_ticker_list=parse_table(sp600cl)
sp600_ticker_list.drop([0],inplace=True)
sp600_ticker_list=sp600_ticker_list[['Symbol\n']]
sp600_ticker_list=sp600_ticker_list.replace(r'\n','', regex=True)
sp600_ticker_list=sp600_ticker_list.rename(columns={'Symbol\n': 'ticker'})
sp600_ticker_list=sp600_ticker_list.reset_index(drop=True)

In [ ]:
'''Do a yfinance.Ticker(ticker).info call of a security to see all available 
features available for a given security.'''
# ticker ='AAP'
# yfinance.Ticker(ticker).info

In [ ]:
# add a feature to your ticker list daataframe for example: 'currentPrice'
tickers=sp600_ticker_list['ticker']
# display(tickers)
spx600 =[]
for ticker in tickers:
    stock = yf.Ticker(ticker)
    try:
        # add as many features as you wish
        if stock.info['currentPrice']:
            spx600.append(ticker + ',' +
                          str(stock.info['currentPrice']))
    except KeyError:
        pass

In [ ]:
spx600pd = pd.Series(spx600)
spx600pd
spx600 = spx600pd.str.split(',', expand=True)
spx600 = spx600.rename(columns={0:'ticker', 1: 'Last'})
### save to an excel file
spx600.to_excel('data/spx600.xlsx')
# analyze your data how you see fit.